In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import themeGPT
from kiwipiepy import Kiwi
import tiktoken
import warnings

openai_key = "발급받은 OPENAI key 입력"

### ThemeGPT-1: 테마명 추출

ThemeGPT-1은 뉴스 기사별로 테마명을 추출하는 프로그램입니다. API 사용량 한정상 전체 코드를 돌릴 경우 과금량이 많아지기 때문에, 코드 수행이 가능한지 판단용으로 3개의 행에 대해서 시행하는 코드를 제공합니다.

In [91]:

# 네이버 테마명 리스트 데이터 : 기존 테마명 예시 제시용
theme_data = pd.read_csv('./data/230804_theme_list_lite.csv', encoding='cp949')

# 전처리 과정을 완료한 뉴스 데이터
news_data = pd.read_excel('./data/news_data_230923-230929.xlsx')
news_data = news_data[['title','date', 'content_preprocess']]


news_data['date'] = news_data['date'].apply(lambda x: x.strftime('%y%m%d'))
news_data['date'] = [int(date) for date in news_data['date']]
news_data.columns = ['TITLE', 'DATE', 'CONTENT']

news_data
# 새로운 열 생성
news_data['THEME'] = ''

# 테마명 리스트에서 괄호 제거 후 유니크한 값만 뽑기
theme = list(set([item.split('(')[0].strip() for item in theme_data.테마명.tolist()]))

# 신규상장 들어간 단어는 하나 제외 제거
# '신규상장'이 포함된 항목 중 가장 숫자가 큰 것을 찾음
latest_new_listing = max([item for item in theme if '신규상장' in item], key=lambda x: int(x.split()[0]))

# 최신 신규상장 항목만 남기고 나머지 항목 제거
theme = [item for item in theme if item == latest_new_listing or '신규상장' not in item]

# 임베딩 모델 로드
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS') 

#인코딩 로드 (토큰 수 카운트 용)
tokenizer = tiktoken.encoding_for_model('gpt-3.5-turbo')


In [88]:
# 일주일 치의 뉴스 데이터 사용. 만약 날짜가 이미 필터링된 뉴스 데이터를 불러올 때 이 부분은 불필요합니다.
# 예시로 여기서는 6월 17일 ~ 6월 23일로 6월 3주차 데이터를 사용합니다.
news_data = news_data[news_data['DATE']>230811].reset_index()

In [89]:
temperature = 0.1

In [92]:
news_data

,TITLE,DATE,CONTENT,THEME
0,"'회생신청' 위니아전자, 체불임금 변제 계획 발표",230926,이란 다야니 측과 공탁금 배당 절차 소송중…신속 진행 촉구멕시코 공장 매각 절차…그...,
1,건설사 자금줄 '숨통' 틔운다…'PF보증 25조+정상화펀드 2조' 투입,230926,HUG·주금공 PF보증 10조 늘려 건설사 대출 지원민간금융사 PF재구조화 펀드 ...,
2,“자강·협력으로 ‘AI 피라미드’ 쌓아 글로벌 AI 컴퍼니 도약”,230926,"SKT, 새로운 AI 전략 공개AI 투자비중 향후 5년간 3배로2028년 관련 매출...",
3,"‘가짜뉴스 논란’ 네이버, 정정보도 서비스 강화(종합)",230926,"정정보도 모음 페이지, 팩트체크 페이지 개편방통위 ‘네이버 뉴스 알고리즘’ 사실조사...",
4,검정 물체가 '스멀스멀'…프랑스 기차타고 가던 관광객들 화들짝 놀라게 한 '빈대',230926,프랑스 기차서 일주일 새 빈대 3건 신고매해 수천만 명이 찾는 관광대국 프랑스가 빈...,
...,...,...,...,...
3085,부산 가덕 신공항 조기 개항 청신호…내년 말 착공 '순조',230928,입법 절차 사실상 마무리…부산 여야 의원 협치 성과 평가24시간 운영가능 남부권 관...,
3086,'30알 1판' 특란이 8000원에 전 부치기 '포기'…고난의 추석 장보기,230928,"전통시장 고사리 150g 5000원, 계란 8000원대8월 소비자물가 3%대 진입…...",
3087,"""나가기 두렵다""…한가위 고물가 파고에 서민들 악소리",230928,휘발유 평균 판매가 1800원대 육박…경윳값도 1700원대 바짝제수용 사과 1개 만...,
3088,"野, 대통령 거부 ‘양곡관리법’ 개정 재추진…뭐가 달라졌나",230929,국회 농해수위에 관련 법안 14건 발의의무매입 대신 '가격 보장제도'가격 일정수준으...,


In [ ]:
# 기존 테마명 임베딩
import openai
from sentence_transformers import SentenceTransformer, util
import time
theme_embedding = model.encode(theme)

# 명령문 프롬프트
order_prompt = """[뉴스 기사]를 분석하여 주식 시장에 영향을 미칠 만한 내용인지 판단한 후, 
가장 관련이 높은 주식 테마 키워드 한 단어와 해당 테마의 연관어 다섯 가지를 추출하세요. 
테마명 추출 예시로는 [기존 테마명 리스트]를 참고하세요."""

for idx, (title, text) in enumerate(zip(news_data["TITLE"], news_data["CONTENT"])):
    if news_data["THEME"][idx] != "":
        continue

    if pd.isna(text):
        continue

    # 뉴스 제목 임베딩
    title = news_data.TITLE[idx]
    title_embedding = model.encode(title)

    # 기존 테마명과 뉴스 제목의 유사도 계산
    similarities = util.cos_sim(title_embedding, theme_embedding)
    similarities = similarities[0].tolist()

    # 뉴스 제목과 유사도가 높은 순으로 100개의 테마명 추출
    sorted_indices = sorted(
        range(len(similarities)), key=lambda i: similarities[i], reverse=True
    )
    selected_indices = sorted_indices[:100]
    theme_selection = [theme[i] for i in selected_indices]

    # 기존 테마명 리스트 제공 프롬프트
    theme_prompt = "[기존 테마명 리스트] \n [" + ", ".join(theme_selection) + "]"

    prompt_default = [
        {
            "role": "system",
            "content": """뉴스 기사의 주요 주식 테마 키워드를 추출하는 프로그램입니다. 이 프로그램은 다음과 같은 단계로 진행합니다.
                STEP 1. 뉴스 기사가 주식 시장에 영향을 미칠 만한 내용을 담고 있는지 판단하세요.
                STEP 2. 그렇지 않다고 판단될 경우, '주식 연관성: 없음'을 출력하고 종료하세요.
                STEP 3. 그렇다고 판단될 경우, '주식 연관성: 있음'을 출력합니다.
                STEP 4. 뉴스 기사와 가장 연관 있는 테마명을 하나 추출합니다. (기존 테마명 리스트에 존재하지 않는 단어도 추출 가능합니다.)
                STEP 5. 기사 내의 정보를 바탕으로 해당 테마명의 연관어 5가지를 추출합니다.
                STEP 6. (출력) '주식 연관성: {}, 주식 테마 키워드: {}, \n 연관어: {}' 형태로 출력합니다.
                
                이 프로그램은 다음 요구사항을 준수합니다.
                  - 기사 내의 정보에만 의존하여 결과를 제공합니다.
                  - 단순히 기사의 주제가 아닌 주식 시장에 영향을 미칠 만한 주식 테마 키워드만을 출력합니다.
                  - 여러 테마 키워드가 감지될 경우, 가장 확률이 높은 테마 키워드 하나만 출력합니다.
                  - 공시자료에서 사용하는 단어를 연관어로 출력합니다.""",
        },
        {"role": "user", "content": theme_prompt},
        {
            "role": "assistant",
            "content": """기존 테마명 리스트를 확인하였습니다. 
                  기존 테마명 리스트에 존재하지 않는 단어도 테마로 추출 가능함을 인지합니다.""",
        },
    ]

    # 텍스트 길이의 토큰이 GPT 제한 초과시 CLOVA SUMMARY API를 통해 뉴스 기사를 10문장으로 요약
    if len(tokenizer.encode(text)) < 2500:
        text = text
    else:
        text = clova_news_summary(text, language="kor", tone=3, summaryCount=10)

    # 프롬프트 완성
    news_prompt = [
        {"role": "user", "content": order_prompt + " \\n [뉴스 기사] \\n" + text}
    ]
    assitant_prompt = [{"role": "assistant", "content": "주식 연관성: "}]
    prompt = prompt_default + news_prompt + assitant_prompt

    openai.api_key = openai_key  # API Key

    # GPT API 호출
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", messages=prompt, temperature=temperature
    )
    news_data["THEME"][idx] = completion["choices"][0]["message"]["content"]
    print(str(idx) + completion["choices"][0]["message"]["content"])

In [105]:
def process_row(row):
    import re
    if '없음' in row:
        return '없음', '', '', '', '', ''
    else:
        theme_match = re.search(r'키워드: (.+?)\n', row)
        keywords_match = re.search(r'연관어: (.+)', row)

        # 매치가 없는 경우를 확인
        if theme_match is None or keywords_match is None:
            return '', '', '', '', '', ''

        theme = theme_match.group(1).strip()
        keywords = [keyword.strip() for keyword in keywords_match.group(1).split(',')]
        keywords += [''] * (5 - len(keywords))  # 연관어가 5개 미만인 경우를 처리
        return theme, keywords[0], keywords[1], keywords[2], keywords[3], keywords[4]


In [106]:
news_data[['테마명', '연관어1', '연관어2', '연관어3', '연관어4', '연관어5']] = news_data['THEME'].apply(lambda row: pd.Series(process_row(row)))

  

In [107]:
news_data

,TITLE,DATE,CONTENT,THEME,테마명,연관어1,연관어2,연관어3,연관어4,연관어5
0,"'회생신청' 위니아전자, 체불임금 변제 계획 발표",230926,이란 다야니 측과 공탁금 배당 절차 소송중…신속 진행 촉구멕시코 공장 매각 절차…그...,"있음\n주식 테마 키워드: 체불임금 변제\n연관어: 이란 다야니, 공탁금 배당 절차...",체불임금 변제,이란 다야니,공탁금 배당 절차,멕시코 공장 매각,대유위니아그룹,회생절차
1,건설사 자금줄 '숨통' 틔운다…'PF보증 25조+정상화펀드 2조' 투입,230926,HUG·주금공 PF보증 10조 늘려 건설사 대출 지원민간금융사 PF재구조화 펀드 ...,"주식 연관성: 있음\n주식 테마 키워드: 건설 대표주\n연관어: 주택공급, 건설사,...",건설 대표주,주택공급,건설사,대출,자금지원,주택도시보증공사
2,“자강·협력으로 ‘AI 피라미드’ 쌓아 글로벌 AI 컴퍼니 도약”,230926,"SKT, 새로운 AI 전략 공개AI 투자비중 향후 5년간 3배로2028년 관련 매출...","있음\n주식 테마 키워드: AI 전략\n연관어: SKT, 인공지능, AI 투자, 매...",AI 전략,SKT,인공지능,AI 투자,매출,데이터센터
3,"‘가짜뉴스 논란’ 네이버, 정정보도 서비스 강화(종합)",230926,"정정보도 모음 페이지, 팩트체크 페이지 개편방통위 ‘네이버 뉴스 알고리즘’ 사실조사...",없음,없음,,,,,
4,검정 물체가 '스멀스멀'…프랑스 기차타고 가던 관광객들 화들짝 놀라게 한 '빈대',230926,프랑스 기차서 일주일 새 빈대 3건 신고매해 수천만 명이 찾는 관광대국 프랑스가 빈...,없음,없음,,,,,
...,...,...,...,...,...,...,...,...,...,...
3085,부산 가덕 신공항 조기 개항 청신호…내년 말 착공 '순조',230928,입법 절차 사실상 마무리…부산 여야 의원 협치 성과 평가24시간 운영가능 남부권 관...,없음,없음,,,,,
3086,'30알 1판' 특란이 8000원에 전 부치기 '포기'…고난의 추석 장보기,230928,"전통시장 고사리 150g 5000원, 계란 8000원대8월 소비자물가 3%대 진입…...",없음,없음,,,,,
3087,"""나가기 두렵다""…한가위 고물가 파고에 서민들 악소리",230928,휘발유 평균 판매가 1800원대 육박…경윳값도 1700원대 바짝제수용 사과 1개 만...,없음,없음,,,,,
3088,"野, 대통령 거부 ‘양곡관리법’ 개정 재추진…뭐가 달라졌나",230929,국회 농해수위에 관련 법안 14건 발의의무매입 대신 '가격 보장제도'가격 일정수준으...,"있음\n주식 테마 키워드: 농업\n연관어: 쌀, 양곡, 가격 보장제도, 양곡관리법,...",농업,쌀,양곡,가격 보장제도,양곡관리법,시장가격


In [108]:
#news_data.to_excel('news_data_230923_230929.xlsx', index = False)

In [5]:

# GPT1을 통한 데이터 출력

# 경고메세지 끄기
warnings.filterwarnings(action='ignore')

news_theme_data = themeGPT.themeGPT1(data = news_data,
                                     theme_data = theme,
                                     temperature = 0.1,
                                     key = openai_key,
                                     embedding_model = model,
                                     tokenizer = tokenizer)

                                     
# 경고메시지 다시 출력하게 하기
warnings.filterwarnings(action='default')

0있음
주식 테마 키워드: 네 마녀의 날
연관어: 준틴스데이, 노예해방일, 단기변동성, 숨 고르기, 쿼드러플 위칭데이
1있음
주식 테마 키워드: 팀스터
연관어: 운송, 물류, 노동자, 조합, 파업
2있음
주식 테마 키워드: 환율하락
연관어: 미국, 재무부, 반기별, 환율, 보고서, 거래, 국가, 수출, 경쟁력, 스위스, 분석, 중국, 대만, 한국, 독일, 말레이시아, 싱가포르, 외환, 경제, 정책, 주의, 모니터링, 일본, 세, 기준, 충족, 제외, 외환 보유액, 판매, 전환, 법률, 개입, 달러, 무역, 이익, 환율 약화, 외환 개입, 엔화, 가치, 폭락, 환율 시장


In [15]:
news_theme_data

,index,Unnamed: 0,DATE,WRITER,TITLE,IMPORTANCE,ITEM_NAME,TAG_LIST,CONTENT,THEME
0,12396,12396,230617,이투데이,"[상보] 뉴욕증시, 연휴 앞두고 ‘숨고르기’에 소폭 하락...다우 0.32%↓",73.08,NaN,NaN,‘네 마녀의 날’ㆍ19일 준틴스데이 앞두고 소폭 하락S%P500지수 5주 연속ㆍ나스...,없음
1,12397,12397,230617,로이터(AI뉴스),"팀스터즈, UPS에서 파업 승인",61.98,NaN,소재 |미국 |운송 |,국제 팀스터 형제단 (International Brotherhood of Teams...,"있음\n주식 테마 키워드: 팀스터\n연관어: 운송, 물류, 노동자, 조합, 파업"
2,12398,12398,230617,로이터(AI뉴스),"미국 재무부, 환율 조작한 거래 파트너 없다고 발표, 스위스 감시 등급 하향 조정",63.69,NaN,일본 |이머징마켓 |대한민국 |독일 |미국 |EU |중국 |아시아 |,미국 재무부는 최근 발표한 반기별 환율 보고서에서 주요 미국 거래 국가 중 환율 조...,"있음\n주식 테마 키워드: 환율하락\n연관어: 미국, 재무부, 반기별, 환율, 보고..."


6월 뉴스기사에 대해 테마명 추출이 완료된 일자를 이후 'news_data_final.xlsx'라는 이름으로 저장하였습니다.

In [ ]:
# news_theme_data.to_excel('./data/news_data_final.xlsx')

### ThemeGPT-2: 테마명 통합

ThemeGPT-1 테마명 추출 프로그램을 통해 뽑은 테마명들 중에서 중복을 제거하는 부분입니다. 실제로 구현할 때는 코드를 수정하여 ThemeGPT-1 과정과 이어서 시행할 수 있지만, GPT API 사용량 한정상 테마 추출을 완료한 뉴스 데이터를 불러와서 사용하도록 하겠습니다. 

In [109]:
# 임베딩 모델 : KR-SBERT
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS') 

# 테마 생성된 뉴스 데이터 불러오기
news_theme_data = pd.read_excel('./data/news_data_final.xlsx')

In [111]:
# 테마 리스트 불러오기

# 1주 단위로 테마명을 통합할 것이므로, 여기서 데이터의 일자 기준으로 필터링을 해줍니다.
# 실제로 구현 시에는 news_theme_data에 일주일 치 데이터를 불러오는 것으로 자동화가 가능합니다.
#news_theme_data = news_theme_data[(news_theme_data.DATE >= 230805) & (news_theme_data.DATE <= 230811)].reset_index(drop = True)
theme_list = news_theme_data['테마명'].tolist()

# 코사인 유사도 기준 유사 단어 그룹 생성
word_groups = themeGPT.generate_word_group(theme_list, #테마 리스트 
                                           embedding_model = model, #임베딩 모델
                                           threshold = 0.75) #단어 그룹을 묶는 코사인 유사도 임계치

In [113]:
word_groups

[{'임금체불', '체불임금 변제'},
 {'건설', '건설 대표주'},
 {'AI', 'AI 전략', 'AI반도체', '로봇', '인공지능', '인공지능/로봇'},
 {'KB금융',
  'KB금융지주',
  '금융',
  '금융권',
  '금융그룹',
  '금융지주',
  '금융지주사',
  '금융지주회사',
  '은행',
  '인터넷은행'},
 {'아웃도어', '아웃도어 의류'},
 {'수출', '수출, 수입', '수출기업', '수출산업'},
 {'스마트 윈도', '스마트 윈도우'},
 {'대형마트', '홈플러스'},
 {'환율하락', '환율하락 수혜'},
 {'가계대출', '대출', '대환대출', '주택담보대출'},
 {'벤처캐피탈', '벤처투자'},
 {'의류', '패션/의류'},
 {'KB금융지주', '금융지주', '금융지주사', '금융지주회사'},
 {'분양',
  '분양주택',
  '빌라',
  '소형아파트',
  '아파트',
  '입주',
  '입주장',
  '주택',
  '주택건설',
  '주택공급',
  '주택공급 활성화',
  '주택공급구조',
  '주택공급대책',
  '주택관리'},
 {'신재생에너지', '재생에너지', '친환경', '친환경 에너지', '친환경주택', '태양광에너지'},
 {'전립선비대증', '전립선비대증 치료제'},
 {'2차전지', '2차전지주', '이차전지', '이차전지주', '전고체 이차전지', '전고체전지'},
 {'카드', '카드론', '카드사'},
 {'생산성 개선', '실적', '실적 개선'},
 {'외식업종', '음식료업종'},
 {'건강보험', '건강보험공단', '건강보험료율'},
 {'재개발', '재개발/재건축', '재건축', '재건축/재개발', '주택정비형 재개발사업'},
 {'국채', '국채금리', '국채수익률', '국채시장'},
 {'게임', '모바일게임'},
 {'분양', '분양주택', '빌라', '소형아파트', '아파트', '입주', '주택'},
 {'파트너', '파트너십', '협력'},
 {'

In [114]:
# ThemeGPT-2를 통한 테마 통합 프로그램 실행
gpt_combined_result = themeGPT.themeGPT2(word_groups, temperature = 0.0, key = openai_key)

In [115]:
print(gpt_combined_result)

['체불임금', '건설', '인공지능', '금융', '아웃도어', '수출', '스마트 윈도우', '홈플러스 대형마트', '환율하락', '대출', '벤처', '의류', '금융지주', '주택공급', '친환경', '전립선비대증', '2차전지', '카드', '실적 개선, 실적', '음식료업종', '건강보험', '재개발', '국채', '게임', '아파트', '파트너', '외환', '준법경영시스템', '비트코인', '상생금융', '디스플레이', '석유화학, 정유', '반도체', '바이오', '금융자동화기기', '챗봇', '대표주', '기계', '바이오', '철강', '탄소', '보험', '스타트업', '가맹', '아프리카돼지열병', '채권', '금리', '회장', '반도체', '플라스틱', '자산운용', '에너지', '우주항공산업', '실적', '현대차', '데이터', '인수목적회사', '금융', '온라인쇼핑몰', '유통', '외제차', '모빌리티', '협력', '업', '금융', '딥테크', '철도', '에너지', '2차전지', '급행철도', '인재경영', '조선', '치료제/분석', '부당거래', '고금리', '신약개발', '성장', '코스닥', '세', '전자책', '배당', '이재명', '보툴리눔', '가치주', '법무장관', '개발사업', '스마트건설', '로봇']


In [116]:

# 경고메세지 끄기
warnings.filterwarnings(action='ignore')

# 테마명 그룹별로 기존의 테마명을 바뀐 테마명으로 통합해줍니다.
for idx, theme in enumerate(news_theme_data['테마명']):
  for group in word_groups:
    if theme in group:
      news_theme_data['테마명'][idx] = gpt_combined_result[word_groups.index(group)]
    else:
      continue

    
# 경고메시지 다시 출력하게 하기
warnings.filterwarnings(action='default')

In [117]:
news_theme_data.head()

,TITLE,DATE,CONTENT,THEME,테마명,연관어1,연관어2,연관어3,연관어4,연관어5
0,"'회생신청' 위니아전자, 체불임금 변제 계획 발표",230926,이란 다야니 측과 공탁금 배당 절차 소송중…신속 진행 촉구멕시코 공장 매각 절차…그...,"있음\n주식 테마 키워드: 체불임금 변제\n연관어: 이란 다야니, 공탁금 배당 절차...",체불임금,이란 다야니,공탁금 배당 절차,멕시코 공장 매각,대유위니아그룹,회생절차
1,건설사 자금줄 '숨통' 틔운다…'PF보증 25조+정상화펀드 2조' 투입,230926,HUG·주금공 PF보증 10조 늘려 건설사 대출 지원민간금융사 PF재구조화 펀드 ...,"주식 연관성: 있음\n주식 테마 키워드: 건설 대표주\n연관어: 주택공급, 건설사,...",건설,주택공급,건설사,대출,자금지원,주택도시보증공사
2,“자강·협력으로 ‘AI 피라미드’ 쌓아 글로벌 AI 컴퍼니 도약”,230926,"SKT, 새로운 AI 전략 공개AI 투자비중 향후 5년간 3배로2028년 관련 매출...","있음\n주식 테마 키워드: AI 전략\n연관어: SKT, 인공지능, AI 투자, 매...",인공지능,SKT,인공지능,AI 투자,매출,데이터센터
3,"‘가짜뉴스 논란’ 네이버, 정정보도 서비스 강화(종합)",230926,"정정보도 모음 페이지, 팩트체크 페이지 개편방통위 ‘네이버 뉴스 알고리즘’ 사실조사...",없음,없음,,,,,
4,검정 물체가 '스멀스멀'…프랑스 기차타고 가던 관광객들 화들짝 놀라게 한 '빈대',230926,프랑스 기차서 일주일 새 빈대 3건 신고매해 수천만 명이 찾는 관광대국 프랑스가 빈...,없음,없음,,,,,


In [118]:
news_theme_data = news_theme_data[news_theme_data['테마명'].apply(lambda x: x not in ('없음', ''))]

In [ ]:
# 테마 통합을 완료한 뉴스 데이터 파일을 다음과 같이 저장합니다.
# news_theme_data.to_excel('news_theme_data_230923_230929_combined.xlsx', index = False)

### ThemeGPT-3: 테마별 뉴스 기사 기준 현재 흐름 요약 및 테마 클러스터명 추출

ThemeGPT-3은 테마 추출과 별도로, 테마 네트워크에서 시각화에 사용됩니다. 테마 네트워크 서비스 '테마빛나비'에서는 각 테마별 AI의 요약을 제시하고, 별도의 알고리즘으로 계산된 테마 클러스터에 대한 명칭을 정해줄 수 있습니다.

In [59]:
# 노드 임베딩 및 클러스터링이 끝난 데이터 호출
theme_nodes_data = pd.read_csv('./data/nodes_sep_week4.csv')

# 뉴스 데이터 호출
# 여기서는 예시로 6월 1주차 데이터를 가져옵니다.
news_theme_data = pd.read_excel('./data/news_theme_data_230923-230929_combined.xlsx')

#theme_nodes_data['AI_summary'] = ''

In [54]:
def AI_summary(theme, news, key, temperature):
    
    # theme : 테마명
    # news: 해당 테마명과 관련된 뉴스 데이터
    # key : OPENAI API key
    # temperature : GPT temperature

    summary_news = [
        clova_news_summary(text, language="ko", tone=2, summaryCount=5) for text in news
    ]

    order_prompt = (
        '아래 리스트로 된 뉴스 기사 3가지를 보고 "' + theme + '" 테마에 대한 이슈와 전망을 5문장 이내로 요약하세요.'
    )

    prompt = [
        {
            "role": "system",
            "content": """
                    - 뉴스 기사의 내용을 바탕으로 5문장 이내로 요약합니다. 
                    - 각 문장은 하오체로 작성합니다. 
                    - 한 문단으로 문장을 이어서 작성합니다.""",
        },
        {"role": "user", "content": order_prompt + "\\n" + str(summary_news)},
    ]

    openai.api_key = key  # API Key
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", messages=prompt, temperature=temperature
    )

    theme_summary = completion["choices"][0]["message"]["content"]

    return theme_summary

In [44]:
import openai
for idx, node in enumerate(theme_nodes_data['node']):

  # GPT 서버 불안정으로 가끔 for문 진행 중 종료되는 경우가 있기 때문에, 이어서 코드를 바로 실행할 수 있도록 if continue문 작성
  if theme_nodes_data["AI_summary"][idx] != '':
    continue

  # 해당 테마가 추출된 기사 중 중요도 기준 상위 3개 뉴스기사 추출
  top3_news = news_theme_data[news_theme_data['테마명'] == node].sample(n=3, random_state=1).CONTENT.tolist()

  # 기사 내용이 비어있지 않은 기사만 사용
  top3_news = [text for text in top3_news if not pd.isna(text)]

  # CLOVA SUMMARY API의 글자 수 제한으로 뉴스 기사의 글자수가 2000이 넘을 경우 w2000까지로 필터링 후 요약
  top3_news = [text[:2000] if len(text) > 2000 else text for text in top3_news]

  # AI의 요약 GPT로 작성
  summary = AI_summary(node, top3_news, temperature = 0.7, key = openai_key)

  # 데이터에 입력
  theme_nodes_data["AI_summary"][idx] = summary 


<ipython-input-44-168c409314b5>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  theme_nodes_data["AI_summary"][idx] = summary
<ipython-input-44-168c409314b5>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  theme_nodes_data["AI_summary"][idx] = summary
<ipython-input-44-168c409314b5>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  theme_nodes_data["AI_summary"][idx] = summary
<ipython-input-44-168c409314b5>:21: SettingWithCopyWarn

In [60]:
# 클러스터 숫자 - 클러스터 이름 사전 생성
cluster_name_dict = {}

cluster_column_name = 'cluster'
for cluster in theme_nodes_data[cluster_column_name].unique():
  
  if cluster == 99:
    cluster_name_dict[99] = '그 외'
    continue
  # 클러스터 하나에 속하는 테마명들 리스트화
  theme_cluster_list = theme_nodes_data[theme_nodes_data[cluster_column_name] == cluster]['node'].tolist()

  # GPT로 클러스터명 추출
  cluster_name_answer = themeGPT.AI_cluster_name(theme_cluster_list, temperature = 0.1, key = openai_key)
  print(cluster_name_answer)
  #흔하게 나오는 오류에 대한 수정

  if '.' in cluster_name_answer:
    # 명사 추출 함수
    def extract_noun(text):
        kiwi = Kiwi()
        nouns = []
        noun = kiwi.analyze(text)
        for token, pos, _, _ in noun[0][0]:
            if len(token) != 1 and pos.startswith('N') or pos.startswith('SL'):
                nouns.append(token)
        return nouns
    
    # 첫명사 추출
    cluster_name_answer = extract_noun(cluster_name_answer)[0]

  #사전에 추가
  cluster_name_dict[cluster] = cluster_name_answer


에너지와 관련된 클러스터
금융 및 건설
스마트폰
항공산업
AI Chat
자동차


In [63]:
cluster_name_dict

{1: '에너지',
 2: '금융 및 건설',
 4: '스마트폰',
 0: '항공산업',
 5: 'AI Chat',
 3: '자동차',
 99: '그 외'}

In [64]:
# 클러스터명이 중복해서 추출되는 경우 그 중에 하나로 통일

# 딕셔너리 반전 및 최소 키 값을 가지는 딕셔너리 생성
reverse_dict = {value: min([k for k, v in cluster_name_dict.items() if v == value]) for value in cluster_name_dict.values()}

# DataFrame 값 치환
theme_nodes_data['cluster'] = theme_nodes_data['cluster'].map(lambda x: reverse_dict.get(cluster_name_dict.get(x, None), x))


In [65]:
# 사전 - 데이터프레임 만들기
cluster_df = pd.DataFrame(list(cluster_name_dict.items()), columns = [cluster_column_name, 'cluster_name'])

# 데이터프레임 조인
cluster_df = pd.merge(theme_nodes_data, cluster_df, on = cluster_column_name, how='inner')

In [66]:
# 데이터 저장
cluster_df.reset_index(drop = True).to_csv('./data/nodes_sep_week4.csv', index = False)